# Gene Regulatory Network 1
***
Here we illustrate how to implement the model describet in [Sturrock M, Hellander A, Matzavinos A, Chaplain MAJ. 2013 Spatial stochastic modelling of the Hes1 gene regulatory network: intrinsic noise can explain heterogeneity in embryonic stem cell differentiation. J R Soc Interface 10:20120988]

This model uses the hes1 mesh (slide 66)
***
## Setup the Environment
***

In [ ]:
import spatialpy

***
## Create the Gene Regulatory Network 1 Model
***

In [ ]:
def create_gene_regulatory_network_1(parameter_values=None):
    model = spatialpy.Model(name="Gene_Regulatory_Network_1")

    model.NUCLEUS = 'Nucleus'
    model.CYTOPLASM = 'Cytoplasm'

    # Domain
    domain = spatialpy.Domain.read_stochss_domain('Gene_Regulatory_Network_1.smdl')
    model.add_domain(domain)

    model.staticDomain = True

    # Variables
    G_F = spatialpy.Species(name="G_F", diffusion_coefficient=0)
    G_O = spatialpy.Species(name="G_O", diffusion_coefficient=0)
    P = spatialpy.Species(name="P", diffusion_coefficient=1e-12)
    mRNA = spatialpy.Species(name="mRNA", diffusion_coefficient=1e-12)
    model.add_species([G_F, G_O, P, mRNA])

    # Initial Conditions
    G_F_ic = spatialpy.PlaceInitialCondition(species="G_F", count=1, location=[0, 0, 0])
    model.add_initial_condition(G_F_ic)

    # Parameters
    mu = spatialpy.Parameter(name="mu", expression="0.002")
    kappa = spatialpy.Parameter(name="kappa", expression="0.015")
    ka = spatialpy.Parameter(name="ka", expression="1.6605778811e-20")
    kd = spatialpy.Parameter(name="kd", expression="0.01")
    gamma_m = spatialpy.Parameter(name="gamma_m", expression="0.0006")
    gamma_p = spatialpy.Parameter(name="gamma_p", expression="0.0003")
    model.add_parameter([mu, kappa, ka, kd, gamma_m, gamma_p])

    # Reactions
    R1 = spatialpy.Reaction(
        name="R1", rate="ka",
        reactants={'G_F': 1, 'P': 1}, products={'G_O': 1}
    )
    R2 = spatialpy.Reaction(
        name="R2", rate="kappa", restrict_to=[model.CYTOPLASM],
        reactants={'mRNA': 1}, products={'mRNA': 1, 'P': 1}
    )
    R3 = spatialpy.Reaction(
        name="R3", rate="mu",
        reactants={'G_F': 1}, products={'G_F': 1, 'mRNA': 1}
    )
    R4 = spatialpy.Reaction(
        name="R4", rate="gamma_m",
        reactants={'mRNA': 1}, products={}
    )
    R5 = spatialpy.Reaction(
        name="R5", rate="gamma_p",
        reactants={'P': 1}, products={}
    )
    R6 = spatialpy.Reaction(
        name="R6", rate="kd",
        reactants={'G_O': 1}, products={'G_F': 1, 'P': 1}
    )
    model.add_reaction([R1, R2, R3, R4, R5, R6])

    # Timespan
    tspan = spatialpy.TimeSpan.arange(43.221610805402705, t=86400, timestep_size=1)
    model.timespan(tspan)
    return model

### Instantiate the Model

In [ ]:
model = create_gene_regulatory_network_1()

***
## Simulation Parameters
***

In [ ]:
def configure_simulation():
    kwargs = {
        # "number_of_trajectories":1,
        # "seed":None,
    }
    return kwargs

***
## Run the Simulation
***

In [ ]:
kwargs = configure_simulation()
results = model.run(**kwargs)

***
## Visualization
***

In [ ]:
results.plot_species('G_F', animated=True, width=None, height=None)